# Lab 12-5 sequence to sequence (Keras + eager version)

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

In [1]:
pip install tensorflow==1.15

  Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os
import pandas as pd
import re
#요약
from gensim.summarization.summarizer import summarize




1.15.0


In [2]:

#path = '/content/drive/My Drive/Colab Notebooks/moviedata2.json'
df = pd.read_json('moviedata2.json')[:10000]


# 전처리

In [123]:
#df = movie_dataframe
def preprocessing(df):
    df.movie_name = df.movie_name.apply(lambda x: x.replace('혻',''))
    
    df.movie_summary = df.movie_summary.apply(lambda x: str(x).lower())
    df.movie_summary = df.movie_summary.apply(lambda x: re.sub("[^A-Za-z. 0-9]", '', x))
    
    df.movie_synopsis = df.movie_synopsis.apply(lambda x : str(x).lower())
    df.movie_synopsis = df.movie_synopsis.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))
    
    df.movie_tagline = df.movie_tagline.apply(lambda x: str(x).lower())
    df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('\\n','').replace('\\xa0',''))
    df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub('[^A-Za-z. 0-9]', '', x))
    df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub(' {2,}', '', x))
    
    #태그라인 없는 데이터 row 삭제
    index = df[df['movie_tagline']==''].index
    df.drop(index, inplace = True)

    #summary 없는 데이터 row 삭제
    summary_index = df[df['movie_summary']==''].index
    df.drop(summary_index, inplace = True)
    
    index = df[df['movie_summary'].str.contains('  ')].index
    df.drop(index , inplace = True)

    df.reset_index()
    
    #summary 문장 수가 너무 많을 경우 summarize를 통해 압축
    for idx, summary in enumerate(df['movie_summary']):
        x = summary.split('. ')
        #print(x)
        if len(x) <= 2:
            df['movie_summary'][idx] = summary
        else:
            df['movie_summary'][idx] = summarize(summary, word_count=150,ratio = 0.3)
        
        
    index = df[df['movie_summary']==''].index
    df.drop(index, inplace=True)
    df.reset_index()
    
    return df

def processed_df(df):
    tl = list(df["movie_tagline"])
    tl = ["".join([ j for j in i if j != "\n"]).strip() for i in tl]

    summ = list(df['movie_summary'])

    df.movie_tagline = tl
    df.movie_summary = summ

    sources = [i.split() for i in df['movie_summary']]
    targets = [i.split() for i in df['movie_tagline']]
    return sources, targets

In [113]:
df.movie_summary[54]

'in a lastditch effort to stop a diabolical plot a dead cia operatives memories secrets and skills are implanted into a deathrow inmate in hopes that he will complete the operatives mission. bill pope ryan reynolds is a cia agent on a mission in london tracking down a shadowy hacker nicknamed the dutchman. when he gets mysteriously ambushed and killed an experimental procedure is used to transfer his memories into dangerous convict jericho stewart kevin costner. when he wakes up with the cia agents memories his mission is to find the dutchman and make the deal with him before the hacker launches icbms and starts world war iii. but complications soon arise and the mission turns personal. captured by a spanish anarchist before he seals a deal with the profitmotivated hacker known only as the dutchman the londonbased cia operative bill pope is killed in the line of duty. suddenly retrieving the memories and the invaluable information inside the brain of the dead agent become the bureaus t

In [120]:
summarize(df.movie_summary[54], word_count=150, ratio = 0.5)

'in a lastditch effort to stop a diabolical plot a dead cia operatives memories secrets and skills are implanted into a deathrow inmate in hopes that he will complete the operatives mission.\nbill pope ryan reynolds is a cia agent on a mission in london tracking down a shadowy hacker nicknamed the dutchman.\nwhen he wakes up with the cia agents memories his mission is to find the dutchman and make the deal with him before the hacker launches icbms and starts world war iii.\ncaptured by a spanish anarchist before he seals a deal with the profitmotivated hacker known only as the dutchman the londonbased cia operative bill pope is killed in the line of duty.\nnow as jericos fragmented memories become inextricably intertwined with popes classified information about the intercontinental ballistic missile system he becomes the target of both the cia and the anarchists.'

# 단어 나누기

In [125]:
sources, targets = processed_df(preprocessing(df))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [127]:
targets

[['the', 'next', 'chapter'],
 ['its', 'alien', 'the', '8th', 'passenger.'],
 ['all',
  'we',
  'have',
  'to',
  'decide',
  'is',
  'what',
  'to',
  'do',
  'with',
  'the',
  'time',
  'that',
  'we',
  'are',
  'given'],
 ['nobody', 'runs', 'forever'],
 ['the', 'magic', 'begins', 'november', '16th.'],
 ['its', 'just', 'a', 'bunch', 'of', 'hocus', 'pocus'],
 ['you',
  'know',
  'his',
  'name.',
  'you',
  'know',
  'the',
  'story.',
  'on',
  'friday',
  'the',
  '13th',
  'witness',
  'his',
  'resurrection...'],
 ['vampires.', 'no', 'interviews.'],
 ['the', 'd', 'is', 'silent.', 'payback', 'wont', 'be.'],
 ['what', 'do', 'you', 'allow', 'to', 'define', 'you'],
 ['90', 'minutes.', '22', 'miles.', 'zero', 'back', 'up.'],
 ['two', 'worlds.', 'one', 'hero.'],
 ['the', 'future', 'begins.'],
 ['...', 'look', 'closer'],
 ['theyre',
  'going',
  'to',
  'pin',
  'something',
  'on',
  'that',
  'smart',
  'cop',
  'from',
  'philidelphia',
  '.',
  '.',
  '.',
  'maybe',
  'a',
  'medal

In [128]:

# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}



In [129]:
# vocabulary for targets
t_vocab = list(set(sum(targets, [])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}




In [16]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    
    if mode == 'source':
        # preprocessing for source (encoder)
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        
        #s_input = [[x for x in each_input if x] for each_input in s_input]
        #pprint(s_input)
        s_len = list(map(lambda sentence : len(sentence), s_input))
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input
    

    elif mode == 'target':
        # preprocessing for target (decoder)
        # input
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        
        #t_input = [[x for x in each_input if x] for each_input in t_input]
        #pprint(t_input)
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
     
        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
        
        return t_len, t_input, t_output

In [17]:
# preprocessing for source
s_max_len = 10
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
#print(s_len, s_input)

In [18]:
# preprocessing for target
t_max_len = 12
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)

[5, 7, 18, 5, 7, 9, 17, 5, 9, 9, 9, 6, 5, 5, 25, 8, 8, 8, 4, 8, 5, 8, 26, 17, 7, 6, 13, 11, 47, 6, 12, 13, 6, 10, 15, 8, 6, 17, 9, 9, 75, 35, 12, 8, 4, 10, 12, 8, 13, 8, 4, 12, 22, 4, 7, 6, 4, 10, 5, 23, 8, 8, 5, 7, 10, 10, 12, 7, 5, 7, 6, 23, 7, 6, 5, 12, 6, 10, 14, 19, 4, 12, 7, 6, 8, 8, 6, 9, 37, 15, 6, 5, 13, 8, 11, 7, 13, 10, 7, 5, 9, 6, 6, 16, 10, 14, 9, 10, 16, 11, 8, 6, 10, 6, 8, 8, 21, 11, 10, 23, 6, 6, 5, 13, 10, 6, 10, 12, 5, 7, 8, 5, 5, 12, 8, 9, 10, 8, 8, 6, 4, 9, 4, 6, 11, 5, 14, 11, 5, 11, 12, 13, 6, 29, 10, 8, 13, 10, 10, 7, 7, 5, 11, 10, 6, 7, 7, 9, 7, 8, 8, 9, 11, 6, 7, 11, 6, 8, 6, 14, 5, 9, 16, 11, 18, 6, 45, 9, 5, 8, 7, 9, 23, 9, 9, 13, 16, 8, 9, 17, 4, 7, 10, 14, 5, 6, 7, 12, 8, 9, 5, 17, 5, 26, 10, 11, 5, 8, 8, 6, 11, 7, 7, 13, 6, 13, 15, 6, 7, 26, 9, 12, 10, 12, 16, 9, 8, 11, 11, 6, 15, 7, 8, 12, 8, 5, 10, 8, 11, 10, 9, 9, 6, 9, 16, 10, 8, 6, 16, 12, 10, 6, 12, 8, 9, 6, 8, 20, 15, 14, 9, 7, 19, 12, 9, 6, 6, 14, 8, 13, 10, 11, 9, 18, 15, 6, 10, 13, 18, 13, 8, 6, 

# hyper-param

In [19]:
# hyper-parameters
epochs = 100
batch_size = 2
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units = 32

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [20]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [21]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [22]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
                
    def call(self, x, hidden, enc_output):
        
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [23]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    
#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))
    
    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)

# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir=checkpoint_dir)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [27]:
#최근 학습된 모델을 불러오고 이어서 EPOCHS로 돌아감
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [24]:
data

<DatasetV1Adapter shapes: ((?,), (?, 10), (?,), (?, 12), (?, 12)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>

In [25]:
EPOCHS = 100

for epoch in range(EPOCHS):
    
    hidden = encoder.initialize_hidden_state()
    #print(hidden.shape)
    total_loss = 0
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        #print(s_input, s_input.shape)
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(s_input, hidden)

            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #Teacher Forcing: feeding the target as the next input
            for t in range(1, t_input.shape[1]):
                
                predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(t_input[:, t], predictions)
            
                dec_input = tf.expand_dims(t_input[:, t], 1) #using teacher forcing
                
        batch_loss = (loss / int(t_input.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradient = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradient, variables))
        
    if epoch % 10 == 0:
        #save model every 10 epoch
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                            total_loss / n_batch,
                                            batch_loss.numpy()))
        checkpoint.save(file_prefix = checkpoint_prefix)

Epoch 0 Loss 195.8770 Batch Loss 4.7306
Epoch 10 Loss 124.6016 Batch Loss 3.1906
Epoch 20 Loss 93.5189 Batch Loss 2.4954
Epoch 30 Loss 72.7913 Batch Loss 1.7229
Epoch 40 Loss 59.3133 Batch Loss 1.3510
Epoch 50 Loss 50.5944 Batch Loss 1.1558
Epoch 60 Loss 44.0625 Batch Loss 1.0846
Epoch 70 Loss 39.6991 Batch Loss 0.8870
Epoch 80 Loss 35.7240 Batch Loss 0.8137
Epoch 90 Loss 32.7558 Batch Loss 0.7465


In [103]:
#restore checkpoint

#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
states = tf.train.get_checkpoint_state(checkpoint_dir)
checkpoint_paths = states.all_model_checkpoint_paths
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [104]:
#sentence = 'I feel hungry'
sentence = df.movie_summary[39]

In [105]:
sentence

'an adaptation of homers great epic the film follows the assault on troy by the united greek forces and chronicles the fates of the men involved. it is the year 1250 b.c. during the late bronze age. two emerging nations begin to clash after paris the trojan prince convinces helen queen of sparta to leave her husband menelaus and sail with him back to troy. after menelaus finds out that his wife was taken by the trojans he asks his brother agamemnon to help him get her back. agamemnon sees this as an opportunity for power. so they set off with 1000 ships holding 50000 greeks to troy. with the help of achilles the greeks are able to fight the never before defeated trojans. but they come to a stop by hector prince of troy. the whole movie shows their battle struggles and the foreshadowing of fate in this remake by wolfgang petersen of homers the iliad.'

In [106]:
df

movie_genre  \
0                  [Adventure, Drama, Horror, Thriller]   
2                                      [Horror, Sci-Fi]   
4                           [Adventure, Drama, Fantasy]   
5                              [Crime, Drama, Thriller]   
6                          [Adventure, Family, Fantasy]   
7                             [Comedy, Family, Fantasy]   
18                          [Horror, Mystery, Thriller]   
21                              [Action, Crime, Horror]   
23                                     [Drama, Western]   
24                                       [Drama, Sport]   
25                        [Action, Adventure, Thriller]   
26                         [Action, Adventure, Fantasy]   
27                          [Action, Adventure, Sci-Fi]   
29                                              [Drama]   
31                    [Crime, Drama, Mystery, Thriller]   
32                            [Action, Crime, Thriller]   
33                   [Action, Crime, Fantasy, Thriller]   
34                  [Action, Adventure, Drama, Romance]   
39                                     [Drama, History]   
44                [Action, Adventure, Sci-Fi, Thriller]   
45                 [Action, Adventure, Drama, Thriller]   
48                    [Biography, Comedy, Drama, Sport]   
49                          [Action, Adventure, Sci-Fi]   
50                   [Action, Horror, Sci-Fi, Thriller]   
51              [Action, Drama, Romance, Thriller, War]   
52                     [Action, Crime, Drama, Thriller]   
54    [Action, Crime, Drama, Mystery, Sci-Fi, Thriller]   
57                        [Adventure, Horror, Thriller]   
58                                 [Drama, Horror, War]   
59                          [Action, Adventure, Sci-Fi]   
...                                                 ...   
9903                   [Comedy, Drama, Family, Romance]   
9904                                   [Action, Horror]   
9906                                           [Comedy]   
9909                                    [Comedy, Drama]   
9911                       [Animation, Comedy, Fantasy]   
9926                                            [Drama]   
9930                   [Documentary, Adventure, Family]   
9931                                    [Comedy, Drama]   
9942                                            [Drama]   
9948                                            [Drama]   
9949  [Musical, Adventure, Comedy, Drama, Romance, W...   
9950                          [Crime, Drama, Film-Noir]   
9953                                          [Western]   
9956                                           [Comedy]   
9960                   [Action, Crime, Drama, Thriller]   
9961                                    [Comedy, Drama]   
9964                             [Drama, Thriller, War]   
9965                                            [Drama]   
9966                                           [Comedy]   
9969                                            [Drama]   
9971                                   [Crime, History]   
9976                                            [Drama]   
9978                              [Drama, Western, War]   
9982                                    [Comedy, Drama]   
9983                            [History, War, Western]   
9984                     [Biography, Drama, Sport, War]   
9986                                            [Drama]   
9988                                            [Drama]   
9991                                           [Comedy]   
9993       [Action, Adventure, Drama, Horror, Thriller]   

                                             movie_name  \
0                               47 Meters Down: Uncaged   
2                                                 Alien   
4     The Lord of the Rings: The Fellowship of the Ring   
5                                              The Mule   
6                 Harry Potter and the Sorcerer's Stone   
7                              

In [107]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
        
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
        
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
   
    for t in range(max_length_targ):
        

        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '
        
        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)    
            
    return result, sentence
    
result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

print(sentence)
print(result)

an adaptation of homers great epic the film follows the assault on troy by the united greek forces and chronicles the fates of the men involved. it is the year 1250 b.c. during the late bronze age. two emerging nations begin to clash after paris the trojan prince convinces helen queen of sparta to leave her husband menelaus and sail with him back to troy. after menelaus finds out that his wife was taken by the trojans he asks his brother agamemnon to help him get her back. agamemnon sees this as an opportunity for power. so they set off with 1000 ships holding 50000 greeks to troy. with the help of achilles the greeks are able to fight the never before defeated trojans. but they come to a stop by hector prince of troy. the whole movie shows their battle struggles and the foreshadowing of fate in this remake by wolfgang petersen of homers the iliad.
fight to save their hands and beauty <eos> 


In [88]:
print(df.movie_tagline[6])  #Troy 8문장 : k = 3
print(df.movie_tagline[23]) #django 39문장 : k = 7

the magic begins november 16th.
the d is silent. payback wont be.


In [92]:
len(df.movie_summary[39].split('. '))

10

In [109]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
        
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    idx = 0
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    
    xList = []
    for t in range(max_length_targ):
      predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
      #print("*"*100)
      #print(predictions)
      #print("*"*100)
      print(tf.argmax(predictions[0]))
      x = tf.nn.top_k(predictions[0], k=3, sorted=True, name=None)
      for i in range(3):
        predicted_id = x[1][i].numpy()


        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
#             return result, sentence
          xList.append([result, sentence])
          result=""

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)  

    return xList

# result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
tList=prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
# print(sentence)
from pprint import pprint
pprint(tList)

tf.Tensor(2633, shape=(), dtype=int64)
tf.Tensor(440, shape=(), dtype=int64)
tf.Tensor(7370, shape=(), dtype=int64)
tf.Tensor(1134, shape=(), dtype=int64)
tf.Tensor(5735, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(212, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(6959, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(5010, shape=(), dtype=int64)
tf.Tensor(4382, shape=(), dtype=int64)
[['fight savage lust and in is undercover red music by over. red remarques '
  'laugh. up. <eos> ',
  'an adaptation of homers great epic the film follows the assault on troy by '
  'the united greek forces and chronicles the fates of the men involved. it is '
  'the year 1250 b.c. during the late bronze age. two emerging nations begin '
  'to clash after paris the trojan prince convinces helen queen of sparta to '
  'leave her husband menelaus and sail with him back to troy. after menelaus '
  'finds out that his wife was taken by the t

In [0]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
#     sentence = preprocess_sentence(sentence)

    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# 새 섹션

# 새 섹션

In [0]:
sentence = 'I feel hungry'


In [0]:
result, sentence, a = evaluate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
print(result)
print(sentence)

ValueError: ignored